In [82]:
from __future__ import annotations

from dataclasses import dataclass
import sys
from typing import Dict, List

In [24]:
@dataclass
class Node:
    value: str
    x: int
    y: int
    prev_node: Node
    next_node: Node

In [25]:
@dataclass
class Graph:
    nodes: list
    start_nodes: list
    end_nodes: list

    def __post_init__(self):
        print('Creating a new graph.')

In [235]:
class Maze():

    # key: prev_dir. Value: dict of square value to new dir
    ACTIONS = {'N': {'S': 'N',
                     'U': 'S',
                     'R': 'E',
                     'L': 'W'},
               'E': {'S': 'E',
                     'U': 'W',
                     'R': 'S',
                     'L': 'N'},
               'S': {'S': 'S',
                     'U': 'N',
                     'R': 'W',
                     'L': 'E'},
               'W': {'S': 'W',
                     'U': 'E',
                     'R': 'N',
                     'L': 'S'},
              }
    
    MOVES = {'N': (0, -1),
             'S': (0, 1),
             'E': (1, 0),
             'W': (-1, 0)
            }
    
#     def build_state_graph(self):
#         """Returns a graph."""
#         nodes = []
#         start_nodes = []
#         end_nodes = []
                
        
#         for y, row in enumerate(self.squares):
#             for x, c in enumerate(row):
#                 node = Node(c, x, y, None, None)
#                 nodes.append(node)
        
#         graph = Graph(nodes, start_nodes, end_nodes)
    
#         return graph
    
    
    def __init__(self, squares):
        """squares is a list of lists of chars"""
        self.squares = squares
        #self.graph = self.build_state_graph()
        
        self.height = len(self.squares)
        self.width = len(self.squares[0])
        
        print(f'Created maze: {self.squares}')
        
    def next_coord(self, x, y, move):
        """move is a tuple of (x_delta, y_delta)
        
        returns (x, y) of new coord, or None if out of bounds.
        """
        new_x, new_y = x + move[0], y+move[1]
        if new_x < 0 or new_x >= self.width or new_y < 0 or new_y >= self.height:
            return None
        return new_x, new_y
        
        
    def steps_to_exit(self, x, y, prev_dir):
        """Returns the # of steps to get from here to the exit '!'. If we reach "X", returns -1.
        
        x, y are the coords of the current square.
        prev_dir is a char, N, S, E, or W, giving direction we went to enter this square."""
        
        args = (x, y, prev_dir)
        if args in self.visited:
            return -1, []
        self.visited.add(args)
        
        cur_sq = self.squares[y][x]
        coord_list = [(x, y)]
        if cur_sq == '!':
            return 0, coord_list
        if cur_sq == 'X':
            return -1, None
        if cur_sq == '?':
            # Evaluate all directions.
            action_list = ['N', 'S', 'E', 'W']
        else:
            action_list = [self.ACTIONS[prev_dir][cur_sq]]

        best_steps = sys.maxsize
        path = None
        for action in action_list:
            move = self.MOVES[action]
            next_sq = self.next_coord(x, y, move)
            if not next_sq:
                continue
            num_steps, path = self.steps_to_exit(next_sq[0], next_sq[1], action)
            
            if num_steps == -1:
                continue
            
            best_steps = min(best_steps, num_steps+1)
        if path:
            return best_steps, coord_list + path
        return -1, None
    
    def try_entrance(self, x, y, prev_dir):
        self.visited = set()
        num_steps, path = self.steps_to_exit(x, y, prev_dir)
        if num_steps > -1:
            print(f'Found soln! x={x}, y={y}, steps={num_steps}, path={path}')
            return True
        return False
    
    def solve_maze(self):
        """Try all entrances."""
        height = self.height
        width = self.width
        
        # Try all entrances from top going S and from bottom going N.
        for x in range(width):
            print(f'from top: x={x}')
            if self.try_entrance(x, 0, 'S'):
                print("yay")
            print('from bottom')
            if self.try_entrance(x, height - 1, 'N'):
                print("yay")
            
        # Try all from left going E and from right going W
        for y in range(height):
            print(f'from left: y={y}')
            if self.try_entrance(0, y, 'E'):
                print("yay")
            print('from right')
            if self.try_entrance(width - 1, y, 'W'):
                print("yay")
        print('No solution')
        return False
            

In [245]:
squares_str = ['LUU?ULXL', 
               'RLRLU!UU',
               'SLRLULXR',
               'UR?RSL?R',
               'RUURRRSL',   #RSL at end
               'S?SLSSLR',
               'RLR?RL?L',
               'LRSRSLRL'
              ]
squares = [[c for c in s] for s in squares_str]

In [246]:
squares

[['L', 'U', 'U', '?', 'U', 'L', 'X', 'L'],
 ['R', 'L', 'R', 'L', 'U', '!', 'U', 'U'],
 ['S', 'L', 'R', 'L', 'U', 'L', 'X', 'R'],
 ['U', 'R', '?', 'R', 'S', 'L', '?', 'R'],
 ['R', 'U', 'U', 'R', 'R', 'R', 'S', 'L'],
 ['S', '?', 'S', 'L', 'S', 'S', 'L', 'R'],
 ['R', 'L', 'R', '?', 'R', 'L', '?', 'L'],
 ['L', 'R', 'S', 'R', 'S', 'L', 'R', 'L']]

In [247]:
maze = Maze(squares)

Created maze: [['L', 'U', 'U', '?', 'U', 'L', 'X', 'L'], ['R', 'L', 'R', 'L', 'U', '!', 'U', 'U'], ['S', 'L', 'R', 'L', 'U', 'L', 'X', 'R'], ['U', 'R', '?', 'R', 'S', 'L', '?', 'R'], ['R', 'U', 'U', 'R', 'R', 'R', 'S', 'L'], ['S', '?', 'S', 'L', 'S', 'S', 'L', 'R'], ['R', 'L', 'R', '?', 'R', 'L', '?', 'L'], ['L', 'R', 'S', 'R', 'S', 'L', 'R', 'L']]


In [248]:
maze.solve_maze()

from top: x=0
from bottom
from top: x=1
from bottom
from top: x=2
from bottom
from top: x=3
from bottom
from top: x=4
from bottom
from top: x=5
from bottom
from top: x=6
from bottom
from top: x=7
from bottom
from left: y=0
from right
from left: y=1
from right
from left: y=2
from right
from left: y=3
from right
from left: y=4
from right
from left: y=5
from right
from left: y=6
from right
from left: y=7
from right
No solution


False

In [249]:
maze.visited

{(7, 7, 'W')}